In [43]:
from math import ceil, log


def draw_seq_bin_tree(nodes, cell_width=2):
    node_width = cell_width + 2
    ws = "·"
    empty_cell = ws * cell_width
    lft_br = f"{(ws * (node_width-2))}╱{ws}"
    rgt_br = f"{ws}╲{(ws * (node_width-2))}"
    cap = f"┌{'─' * cell_width}┐"
    base = f"└{'─' * cell_width}┘"
    space_width = 2

    n = len(nodes)
    nodes = [str(node).zfill(cell_width) for node in nodes]
    tree_height = ceil(log(n + 1, 2))
    max_lvl_nodes = 2 ** (tree_height - 1)
    max_lvl_gaps = max_lvl_nodes - 1
    last_lvl_gap_w = node_width

    max_box_width = (max_lvl_nodes * node_width) + (max_lvl_gaps * last_lvl_gap_w)
    row_width = max_box_width + (space_width * 3)
    rows_n = (tree_height * 4) + 1
    empty_row = ws * (row_width+(cell_width))
    vis_lst = []
    for row_i in range(rows_n):
        if row_i == 0 or row_i == rows_n - 1:
            vis_lst.append(empty_row)
        else:
            lvl_i = row_i // 4
            lvl_slots_n = 2 ** lvl_i
            lvl_gaps_n = lvl_slots_n - 1
            lvl_start = lvl_slots_n - 1
            lvl_stop = (lvl_start * 2) + 1

            lvl_gap_w = (2 ** (tree_height + 2 - lvl_i)) - node_width
            lvl_box_w = (lvl_slots_n * node_width) + (lvl_gaps_n * lvl_gap_w)
            lvl_margin_w = ((row_width - lvl_box_w) // 2) + 1
            lvl_margin = ws * lvl_margin_w
            lvl_gap = ws * lvl_gap_w

            lvl_fill = ""
            if lvl_i + 1 == tree_height:
                max_slots = (2 ** tree_height) - 1
                missing_n = max_slots - n
                lvl_fill_w = (missing_n * node_width) + (missing_n * lvl_gap_w)
                lvl_fill = ws * lvl_fill_w

            lvl_nodes = [f"│{node}│" for node in nodes[lvl_start:lvl_stop]]
            lvl_legend = empty_cell

            if row_i % 4 == 0:
                # branch row
                lvl_nodes = [lft_br if br % 2 == 0 else rgt_br for br in range(len(lvl_nodes))]
            elif (row_i + 1) % 4 == 0:
                # base row
                lvl_nodes = [base for _ in lvl_nodes]
            elif (row_i + 3) % 4 == 0:
                # cap row
                lvl_nodes = [cap for _ in lvl_nodes]
            else:
                lvl_legend = str(lvl_i).zfill(cell_width)
            vis_lst.append(f"{lvl_legend}{lvl_margin[cell_width:]}{lvl_gap.join(lvl_nodes)}{lvl_fill}{lvl_margin}")
    return vis_lst

In [49]:
import os
import time
from collections import deque
import socket
from pathlib import Path

class BSTNode:
    def __init__(self, val=None):
        self.left = None
        self.right = None
        self.val = val

    def insert(self, val):
        if not self.val:
            self.val = val
            return

        if self.val == val:
            return

        if val < self.val:
            if self.left:
                self.left.insert(val)
                return
            self.left = BSTNode(val)
            return

        if self.right:
            self.right.insert(val)
            return
        self.right = BSTNode(val)

    def get_min(self):
        current = self
        while current.left is not None:
            current = current.left
        return current.val

    def get_max(self):
        current = self
        while current.right is not None:
            current = current.right
        return current.val

    def delete(self, val):
        if self == None:
            return self
        if self.right == None:
            return self.left
        if self.left == None:
            return self.right
        if val < self.val:
            if self.left:
                self.left = self.left.delete(val)
            return self
        if val > self.val:
            if self.right:
                self.right = self.right.delete(val)
            return self
        min_larger_node = self.right
        while min_larger_node.left:
            min_larger_node = min_larger_node.left
        self.val = min_larger_node.val
        self.right = self.right.delete(min_larger_node.val)
        return self

    def exists(self, val):
        if val == self.val:
            return True

        if val < self.val:
            if self.left == None:
                return False
            return self.left.exists(val)

        if self.right == None:
            return False
        return self.right.exists(val)

    def preorder(self, vals):
        if self.val is not None:
            vals.append(self.val)
        if self.left is not None:
            self.left.preorder(vals)
        if self.right is not None:
            self.right.preorder(vals)
        return vals

    def inorder(self, vals):
        if self.left is not None:
            self.left.inorder(vals)
        if self.val is not None:
            vals.append(self.val)
        if self.right is not None:
            self.right.inorder(vals)
        return vals

    def postorder(self, vals):
        if self.left is not None:
            self.left.postorder(vals)
        if self.right is not None:
            self.right.postorder(vals)
        if self.val is not None:
            vals.append(self.val)
        return vals

def main():
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind(('localhost', 12349))
    s.listen(5)
    conn, addr = s.accept()
    current_time = time.strftime("%d-%m-%Y_%H-%M-%S", time.localtime())
    directory_name = os.path.join(Path.cwd(), current_time)
    os.makedirs(directory_name)
    
    print("Enter numbers ('q' to stop):")
    numbers = []
    while True:
        num = conn.recv(1024)
        if num.decode() == 'q':
            break
        else:
            numbers.append(int(num.decode()))
            file_name = os.path.join(directory_name, f"{num.decode()}.json")
            with open(file_name, "w") as file:
                pass
    bst = BSTNode()
    visualisation_rows = draw_seq_bin_tree(numbers)
    for i in numbers:
        bst.insert(i)
    with open(f'{directory_name}\\binary_tree.txt', "w") as file:
        file.write(f'Inorder - {bst.inorder([])}\n')
        file.write(f'Postorder - {bst.postorder([])}\n')
        file.write(f'Preorder - {bst.preorder([])}\n')
        for j in range(len(visualisation_rows)):
            file.write(visualisation_rows[j] + "\n")
    conn.close()

if __name__ == "__main__":
    main()

Enter numbers ('q' to stop):
